In [ ]:
import oandapyV20
import oandapyV20.endpoints.instruments as instruments
from datetime import datetime, timedelta
import pandas as pd
import oandapyV20.endpoints.trades as trades
import json
account_id = ""
api_token = ""
api_url="api-fxpractice.oanda.com"

# filepath: /Users/vidhulkhanna/trading-test/api_test.ipynb
client = oandapyV20.API(
  access_token=api_token,
  environment="practice"            # ← live vs practice
)

In [5]:
import oandapyV20
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.accounts as accounts
import oandapyV20.endpoints.instruments as instruments

def place_market_order(client, account_id, instrument, units, stop_loss_price=None, take_profit_price=None):
    """
    Place a market order with optional stop loss and take profit
    
    Args:
        client: OANDA API client
        account_id: Your account ID
        instrument: Trading instrument (e.g., 'EUR_USD')
        units: Number of units (positive for buy, negative for sell)
        stop_loss_price: Stop loss price level
        take_profit_price: Take profit price level
    """
    
    # Build the order data
    order_data = {
        "order": {
            "type": "MARKET",
            "instrument": instrument,
            "units": str(units),
            "timeInForce": "FOK"  # Fill or Kill
        }
    }
    
    # Add stop loss if provided
    if stop_loss_price:
        order_data["order"]["stopLossOnFill"] = {
            "price": str(stop_loss_price)
        }
    
    # Add take profit if provided
    if take_profit_price:
        order_data["order"]["takeProfitOnFill"] = {
            "price": str(take_profit_price)
        }
    
    # Create and execute the order
    r = orders.OrderCreate(account_id, data=order_data)
    
    try:
        response = client.request(r)
        print("Order placed successfully!")
        print(f"Order ID: {response['orderCreateTransaction']['id']}")
        return response
    except Exception as e:
        print(f"Error placing order: {e}")
        return None

# Example: Buy 1000 units of EUR_USD with stop loss and take profit
instrument = "EUR_USD"
units = 1000  # Positive for buy, negative for sell
stop_loss = 1.14  # Stop loss price
take_profit = 1.16  # Take profit price

# print current price
params = {
    "count": 1,
    "granularity": "S5"  # 5-second granularity for current price
}
r = instruments.InstrumentsCandles(instrument=instrument, params=params)
client.request(r)
current_price = r.response['candles'][0]['mid']['c']
print(f"Current price of {instrument}: {current_price}")
# Place the order
response = place_market_order(
    client=client,
    account_id=account_id,
    instrument=instrument,
    units=units,
    stop_loss_price=stop_loss,
    take_profit_price=take_profit
)
print(response)

Current price of EUR_USD: 1.15803
Order placed successfully!
Order ID: 1189
{'orderCreateTransaction': {'id': '1189', 'accountID': '101-004-6156530-002', 'userID': 6156530, 'batchID': '1189', 'requestID': '61383645904863174', 'time': '2025-06-16T18:49:08.063589684Z', 'type': 'MARKET_ORDER', 'instrument': 'EUR_USD', 'units': '1000', 'timeInForce': 'FOK', 'positionFill': 'DEFAULT', 'takeProfitOnFill': {'price': '1.16000', 'timeInForce': 'GTC'}, 'stopLossOnFill': {'price': '1.14000', 'timeInForce': 'GTC', 'triggerMode': 'TOP_OF_BOOK'}, 'reason': 'CLIENT_ORDER'}, 'orderFillTransaction': {'id': '1190', 'accountID': '101-004-6156530-002', 'userID': 6156530, 'batchID': '1189', 'requestID': '61383645904863174', 'time': '2025-06-16T18:49:08.063589684Z', 'type': 'ORDER_FILL', 'orderID': '1189', 'instrument': 'EUR_USD', 'units': '1000', 'requestedUnits': '1000', 'price': '1.15811', 'pl': '0.0000', 'quotePL': '0', 'financing': '0.0000', 'baseFinancing': '0', 'commission': '0.0000', 'accountBalance

In [12]:
import oandapyV20.endpoints.positions as positions

def get_all_positions(client, account_id):
    """
    Get all positions (both open and closed)
    """
    r = positions.PositionList(account_id)
    response = client.request(r)
    return response

def get_open_positions(client, account_id):
    """
    Get only open positions
    """
    r = positions.OpenPositions(account_id)
    response = client.request(r)
    return response

def display_positions(positions_data):
    """
    Display positions in a readable format
    """
    if not positions_data['positions']:
        print("No open positions found.")
        return
    
    print("\n=== OPEN POSITIONS ===")
    for pos in positions_data['positions']:
        instrument = pos['instrument']
        long_units = float(pos['long']['units'])
        short_units = float(pos['short']['units'])
        net_units = long_units + short_units
        
        if net_units != 0:
            print(f"\nInstrument: {instrument}")
            print(f"Net Units: {net_units}")
            print(f"Long Units: {long_units}")
            print(f"Short Units: {short_units}")
            
            if long_units > 0:
                print(f"Long P&L: {pos['long']['pl']}")
                print(f"Long Unrealized P&L: {pos['long']['unrealizedPL']}")
            
            if short_units < 0:
                print(f"Short P&L: {pos['short']['pl']}")
                print(f"Short Unrealized P&L: {pos['short']['unrealizedPL']}")

# Using your existing connection
client = oandapyV20.API(access_token=api_token)

# Get and display open positions
open_positions = get_open_positions(client, account_id)
display_positions(open_positions)

# Alternative: Get all positions (including closed ones)
all_positions = get_all_positions(client, account_id)
print(f"\nTotal positions: {len(all_positions['positions'])}")


=== OPEN POSITIONS ===

Instrument: EUR_USD
Net Units: 3100.0
Long Units: 3100.0
Short Units: 0.0
Long P&L: -2.7000
Long Unrealized P&L: -0.6880

Total positions: 8


In [13]:
import oandapyV20.endpoints.positions as positions

def close_all_positions(client, account_id):
    """
    Close all open positions
    """
    # Get all open positions
    r = positions.OpenPositions(account_id)
    response = client.request(r)
    
    if not response['positions']:
        print("No open positions to close.")
        return
    
    print("Closing all open positions...")
    
    for pos in response['positions']:
        instrument = pos['instrument']
        long_units = float(pos['long']['units'])
        short_units = float(pos['short']['units'])
        
        # Close long position if exists
        if long_units > 0:
            close_data = {
                "longUnits": "ALL"
            }
            r = positions.PositionClose(account_id, instrument, data=close_data)
            try:
                close_response = client.request(r)
                print(f"Closed long position in {instrument}: {long_units} units")
                print(f"P&L: {close_response['longOrderFillTransaction']['pl']}")
            except Exception as e:
                print(f"Error closing long position in {instrument}: {e}")
        
        # Close short position if exists
        if short_units < 0:
            close_data = {
                "shortUnits": "ALL"
            }
            r = positions.PositionClose(account_id, instrument, data=close_data)
            try:
                close_response = client.request(r)
                print(f"Closed short position in {instrument}: {short_units} units")
                print(f"P&L: {close_response['shortOrderFillTransaction']['pl']}")
            except Exception as e:
                print(f"Error closing short position in {instrument}: {e}")

def close_specific_position(client, account_id, instrument, side="ALL"):
    """
    Close a specific position
    
    Args:
        client: OANDA API client
        account_id: Your account ID
        instrument: Trading instrument (e.g., 'EUR_USD')
        side: "ALL", "LONG", or "SHORT"
    """
    close_data = {}
    
    if side.upper() == "ALL":
        close_data = {
            "longUnits": "ALL",
            "shortUnits": "ALL"
        }
    elif side.upper() == "LONG":
        close_data = {
            "longUnits": "ALL"
        }
    elif side.upper() == "SHORT":
        close_data = {
            "shortUnits": "ALL"
        }
    
    r = positions.PositionClose(account_id, instrument, data=close_data)
    
    try:
        response = client.request(r)
        print(f"Successfully closed {side} position in {instrument}")
        return response
    except Exception as e:
        print(f"Error closing {side} position in {instrument}: {e}")
        return None

# Using your existing connection
client = oandapyV20.API(access_token=api_token)

# First, let's see what positions we have
print("=== CURRENT POSITIONS ===")
open_positions = get_open_positions(client, account_id)
display_positions(open_positions)

# Close all positions
close_all_positions(client, account_id)

# Verify positions are closed
print("\n=== POSITIONS AFTER CLOSING ===")
open_positions_after = get_open_positions(client, account_id)
display_positions(open_positions_after)

=== CURRENT POSITIONS ===

=== OPEN POSITIONS ===

Instrument: EUR_USD
Net Units: 3100.0
Long Units: 3100.0
Short Units: 0.0
Long P&L: -2.7000
Long Unrealized P&L: -0.6880
Closing all open positions...
Closed long position in EUR_USD: 3100.0 units
P&L: -0.6880

=== POSITIONS AFTER CLOSING ===
No open positions found.
